## Edit the bash array in the next cell and run it to choose which notebook-based tests to run:

In [ ]:
test_items=(
    ci-check.ipynb
)

## Run next cell to run each test off-screen:

In [ ]:
for atest in "${test_items[@]}"; do
    if [ -f "$atest" ] && \
        rm -f "$atest.tmp" && \
        cp "$atest" "$atest.tmp" ; then
            echo "Running $i..."
            time jupyter nbconvert --execute  --inplace  --ExecutePreprocessor.timeout=-1 "$atest.tmp"
            echo "   ...finished $i."
    else
        echo "Test $atest not found."
    fi
done

## Run the next cell to test out jq!!

In [ ]:
for atest in "${test_items[@]}"; do
    if [ -f "$atest.tmp" ] ; then
       jq '(.cells[].outputs)' "$atest.tmp"
       jq '(.cells[].outputs)' "$atest.tmp" | grep -qF '[[FAILED]]' && echo "something failed!!"
    else
        echo "Result for test $atest not found."
    fi
done